In [1]:
import signac
import panedr
import numpy as np
import unyt as u

In [2]:
project = signac.get_project()

In [3]:
pretty_names = {
    "cassandra": "Cassandra",
    "mcccs": "MCCCS-MN",
    "gomc": "GOMC",
    "gromacs": "GROMACS",
    "hoomd": "HOOMD-blue",
    "lammps-VU": "LAMMPS",
}
pretty_molecules = {
    "methaneUA": "TraPPE Methane",
    "pentaneUA": "TraPPE Pentane",
    "benzeneUA": "TraPPE Benzene",
    "ethanolAA": "OPLS-AA Ethanol",
    "waterSPCE": "SPC/E Water"}

In [4]:
engines = ["lammps-VU",
           "hoomd",
           "gromacs",
           "mcccs",
           "gomc",
           "cassandra"]

molecules = ["methaneUA", "pentaneUA", "benzeneUA", "waterSPCE", "ethanolAA"]
properties = ['potential_energy',
        'tot_vdw_energy',
        'tail_energy',
        'tot_electrostatics',
        'short_range_electrostatics',
        'long_range_electrostatics',
        'tot_pair_energy',
        'bonds_energy',
        'angles_energy',
        'dihedrals_energy',
        'tot_bonded_energy',
        'intramolecular_energy',
        'intermolecular_energy',]

In [5]:
spe_data = dict()
for molecule in molecules:
    spe_data[molecule] = dict()
    for engine in engines:
        spe_data[molecule][engine] = dict()
        for property in properties:
            spe_data[molecule][engine][property] = 0

# Parse LAMMPS SPE

In [6]:
lmp_map = {"potential_energy": "potential_energy",
           "tot_vdw_energy": "tot_vdw_energy",
           "short_range_electrostatics": "tot_electrostatics",
           "tot_electrostatics": "tot_electrostatics",
           "pair_energy": "tot_pair_energy",
           "bonds_energy": "bonds_energy",
           "angles_energy": "angles_energy",
           "dihedrals_energy": "dihedrals_energy",
           "tail_energy": "tail_energy",
           "long_range_electrostatics": "long_range_electrostatics",}

In [7]:
for job in project.find_jobs({"engine": "lammps-VU"}):
    data = np.genfromtxt(f"{job.ws}/log-spe.txt", names=True, delimiter=",")
    for prop in data.dtype.names:
        if prop in properties:
            spe_data[job.sp.molecule][job.sp.engine][prop] += data[prop]
        else:
            print(f"Not included {job.sp.molecule} {prop}")

Not included ethanolAA pair_energy
Not included pentaneUA pair_energy
Not included benzeneUA pair_energy
Not included methaneUA pair_energy
Not included waterSPCE pair_energy


In [8]:
# for job in project.find_jobs({"engine": "lammps-VU"}):
#     data = np.genfromtxt(f"{job.ws}/log-spe.txt", names=True, delimiter=",")
#     for prop in data.dtype.names:
#         spe_data[job.sp.molecule][job.sp.engine][lmp_map[prop]] += data[prop]

# Parse HOOMD SPE

In [9]:
properties = ['potential_energy',
        'tot_vdw_energy',
        'tail_energy',
        'tot_electrostatics',
        'short_range_electrostatics',
        'long_range_electrostatics',
        'tot_pair_energy',
        'bonds_energy',
        'angles_energy',
        'dihedrals_energy',
        'tot_bonded_energy',
        'intramolecular_energy',
        'intermolecular_energy',]

In [10]:
hoomd_map = {"pair_LJ": ("tot_pair_energy", "tot_vdw_energy",),
             "pair_LJ_tail": "tail_energy",
             "pair_Ewald": ("short_range_electrostatics", "tot_electrostatics",),
             "pppm_Coulomb": ("long_range_electrostatics", "tot_electrostatics",),
             "special_pair_LJ": "tot_pair_energy",
             "special_pair_Coulomb": ("short_range_electrostatics", "tot_electrostatics"),
             "bond_Harmonic": "bonds_energy",
             "angle_Harmonic": "angles_energy",
             "dihedral_OPLS": "dihedrals_energy",
             "potential_energy": "potential_energy"}

In [11]:
for job in project.find_jobs({"engine": "hoomd"}):
    data = np.genfromtxt(f"{job.ws}/log-spe-raw.txt", names=True, delimiter=" ")
    for prop in data.dtype.names:
        if isinstance(hoomd_map[prop], str):
            spe_data[job.sp.molecule][job.sp.engine][hoomd_map[prop]] += data[prop]
        else:
            for key in hoomd_map[prop]:
                spe_data[job.sp.molecule][job.sp.engine][key] += data[prop]


# Parse GROMACS SPE


In [12]:
for job in project.find_jobs({"engine": "gromacs"}):
    data = np.genfromtxt(f"{job.ws}/log-spe-p3m.txt", names=True, delimiter=",")
    for prop in data.dtype.names:
        if prop in properties:
            spe_data[job.sp.molecule][job.sp.engine][prop] += data[prop]
        else:
            print(f"Not included {job.sp.molecule} {prop}")

Not included benzeneUA lj14
Not included benzeneUA coulomb14
Not included methaneUA lj14
Not included methaneUA coulomb14
Not included waterSPCE lj14
Not included waterSPCE coulomb14
Not included pentaneUA lj14
Not included pentaneUA coulomb14
Not included ethanolAA lj14
Not included ethanolAA coulomb14


# Parse MCCCS SPE

In [13]:
for job in project.find_jobs({"engine": "mcccs"}):
    data = np.genfromtxt(f"{job.ws}/log-spe.txt", names=True, delimiter=",")
    for prop in data.dtype.names:
        if prop in properties:
            spe_data[job.sp.molecule][job.sp.engine][prop] += data[prop]
        else:
            print(f"Not included {job.sp.molecule} {prop}")

# Parse Cassandra SPE

In [14]:
for job in project.find_jobs({"engine": "cassandra"}):
    data = np.genfromtxt(f"{job.ws}/log-spe.txt", names=True, delimiter=",")
    for prop in data.dtype.names:
        if prop in properties:
            spe_data[job.sp.molecule][job.sp.engine][prop] += data[prop]
        else:
            print(f"Not included {job.sp.molecule} {prop}")

# Create log-spe with full column from GOMC

In [15]:
# get_e_titles = bool as there are 2 headers, and it will print both otherwise
# log_file_splitline_iter[1] == '0': means box 0 (typicially liquid),  1= box 1 (typically vapor)

In [16]:
def K_to_kj_per_mol_conversion(value):
    conversion_factor = (1 * u.Kelvin).to_value("kJ/mol", equivalence="thermal") #conversion_K_to_kj_per_mol
    converted = value * conversion_factor
    return converted

In [17]:
gomc_map = {
    "TOTAL": "potential_energy",
    "INTRA(B)": "intramolecular_energy",
    "INTRA(NB)": ("intramolecular_energy", "tot_vdw_energy", "tot_pair_energy"),
    "BOND(B)": "bonds_energy",
    "ANGLE(B)": "angles_energy",
    "DIHEDRAL(B)": "dihedrals_energy",
    "INTER(LJ)": ("intermolecular_energy", "tot_vdw_energy", "tot_pair_energy"),
    "LRC": ("tail_energy", "tot_vdw_energy"),
    "TOTAL_ELECT": "tot_electrostatics",
}

In [18]:
for job in project.find_jobs({"engine": "gomc"}):
    with open(f"{job.ws}/out_production_run.dat") as f:
        data = f.readlines()
    for i, line in enumerate(data):
        if "INITIAL SIMULATION ENERGY" in line:
            for title, value in zip(data[i+2].split(), data[i+4].split()):
                 # spe_data[job.id][title] = value
                if title in gomc_map:
                    if isinstance(gomc_map[title], str):
                        spe_data[job.sp.molecule][job.sp.engine][gomc_map[title]] += K_to_kj_per_mol_conversion(float(value))
                    else:
                        for key in gomc_map[title]:
                            spe_data[job.sp.molecule][job.sp.engine][key] += K_to_kj_per_mol_conversion(float(value))
                else:
                    print(f"Not included {job.sp.molecule} {title}")

Not included benzeneUA ETITLE:
Not included benzeneUA STEP
Not included benzeneUA REAL
Not included benzeneUA RECIP
Not included benzeneUA SELF
Not included benzeneUA CORR
Not included ethanolAA ETITLE:
Not included ethanolAA STEP
Not included ethanolAA REAL
Not included ethanolAA RECIP
Not included ethanolAA SELF
Not included ethanolAA CORR
Not included waterSPCE ETITLE:
Not included waterSPCE STEP
Not included waterSPCE REAL
Not included waterSPCE RECIP
Not included waterSPCE SELF
Not included waterSPCE CORR
Not included methaneUA ETITLE:
Not included methaneUA STEP
Not included methaneUA REAL
Not included methaneUA RECIP
Not included methaneUA SELF
Not included methaneUA CORR
Not included pentaneUA ETITLE:
Not included pentaneUA STEP
Not included pentaneUA REAL
Not included pentaneUA RECIP
Not included pentaneUA SELF
Not included pentaneUA CORR


# Summary

In [19]:
import pandas as pd

summarize_dfs = dict()
for molecule in spe_data:
    print(molecule)
    df = pd.DataFrame.from_dict(spe_data[molecule]).transpose()
    display(df)
    df.to_csv(f"csvs/{molecule}_spe.csv")

methaneUA


,potential_energy,tot_vdw_energy,tail_energy,tot_electrostatics,short_range_electrostatics,long_range_electrostatics,tot_pair_energy,bonds_energy,angles_energy,dihedrals_energy,tot_bonded_energy,intramolecular_energy,intermolecular_energy
lammps-VU,536743.553773,536743.553773,-128.231235,0.0,0.0,0.0,536743.553773,0.0,0.0,0.0,0.0,NaN,NaN
hoomd,536736.470216,536736.470216,-128.229542,0.0,0.0,0.0,536736.470216,0.0,0.0,0.0,0.0,0.0,0.000000
gromacs,536736.437500,536736.442657,-128.244843,0.0,0.0,0.0,536736.442657,0.0,0.0,0.0,0.0,NaN,NaN
mcccs,536736.674114,536736.674114,-128.229591,0.0,NaN,NaN,536736.674114,0.0,0.0,0.0,0.0,NaN,NaN
gomc,536736.554367,536736.553776,-128.229552,0.0,0.0,0.0,536864.783328,0.0,0.0,0.0,0.0,0.0,536864.783328
cassandra,536737.280000,536737.280260,-128.229740,0.0,0.0,0.0,536737.280260,NaN,0.0,0.0,NaN,0.0,536865.510000


pentaneUA


,potential_energy,tot_vdw_energy,tail_energy,tot_electrostatics,short_range_electrostatics,long_range_electrostatics,tot_pair_energy,bonds_energy,angles_energy,dihedrals_energy,tot_bonded_energy,intramolecular_energy,intermolecular_energy
lammps-VU,537739.120141,537739.120141,-181.108084,0.0,0.0,0.0,537739.120141,5.538203e-07,6.262974e-08,2.922458e-09,6.193725e-07,NaN,NaN
hoomd,537737.484007,537737.484000,-181.107804,0.0,0.0,0.0,537737.484000,5.538199e-07,6.686200e-06,2.921942e-09,0.000000e+00,0.000000,0.000000
gromacs,537737.187500,537737.164017,-181.148483,0.0,0.0,0.0,537737.164017,2.275352e-06,6.813034e-06,5.722046e-03,5.731134e-03,NaN,NaN
mcccs,537737.792781,537737.792781,-181.107910,0.0,NaN,NaN,537737.792781,0.000000e+00,6.262988e-08,2.921697e-09,6.555158e-08,NaN,NaN
gomc,537737.391951,537737.381481,-181.107817,0.0,0.0,0.0,537918.489298,2.448629e-06,7.013629e-06,1.004162e-02,0.000000e+00,-122.090272,538040.589622
cassandra,537738.400000,537738.401390,-181.108110,0.0,0.0,0.0,537738.401390,NaN,6.370289e-08,2.935907e-09,NaN,-122.100500,538041.610000


benzeneUA


,potential_energy,tot_vdw_energy,tail_energy,tot_electrostatics,short_range_electrostatics,long_range_electrostatics,tot_pair_energy,bonds_energy,angles_energy,dihedrals_energy,tot_bonded_energy,intramolecular_energy,intermolecular_energy
lammps-VU,388924.223641,388924.223579,-250.549162,0.0,0.0,0.0,388924.223579,6.028773e-05,1.522308e-06,3.992398e-08,6.184996e-05,NaN,NaN
hoomd,388939.009526,388939.009526,-250.558687,0.0,0.0,0.0,388939.009526,1.536153e-22,2.882556e-10,0.000000e+00,0.000000e+00,0.000000e+00,0.000000
gromacs,388939.125000,388939.131805,-250.586945,0.0,0.0,0.0,388939.131805,0.000000e+00,3.852421e-10,0.000000e+00,3.852421e-10,NaN,NaN
mcccs,388939.009581,388939.009519,-250.558687,0.0,NaN,NaN,388939.009519,6.028811e-05,1.522316e-06,3.992194e-08,6.185035e-05,NaN,NaN
gomc,388939.051080,388939.051186,-250.558706,0.0,0.0,0.0,389189.609891,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,389189.609891
cassandra,388939.820000,388939.821030,-250.558970,0.0,0.0,0.0,388939.821030,NaN,3.306836e-07,7.699150e-08,NaN,4.076752e-07,389190.380000


waterSPCE


,potential_energy,tot_vdw_energy,tail_energy,tot_electrostatics,short_range_electrostatics,long_range_electrostatics,tot_pair_energy,bonds_energy,angles_energy,dihedrals_energy,tot_bonded_energy,intramolecular_energy,intermolecular_energy
lammps-VU,57176.385301,76663.713512,-275.651013,-19487.328212,292697.838098,-312185.166310,57176.385301,0.0,0.000000e+00,0.0,0.0,NaN,NaN
hoomd,57176.472998,76663.760676,-275.651183,-19487.287483,-18732.078823,-755.208660,76663.760676,0.0,0.000000e+00,0.0,0.0,0.000000e+00,0.000000
gromacs,57175.917969,76663.549835,-275.825165,-19487.629883,0.000000,4493.100586,57175.919952,0.0,0.000000e+00,0.0,0.0,NaN,NaN
mcccs,57175.422752,76664.198289,-275.652756,-19488.775537,NaN,NaN,57175.422752,0.0,0.000000e+00,0.0,0.0,NaN,NaN
gomc,57175.433264,76663.767176,-275.651204,-19488.333923,0.000000,0.000000,76939.418379,0.0,0.000000e+00,0.0,0.0,0.000000e+00,76939.418379
cassandra,57175.778000,76663.843520,-275.651480,-19488.070300,324948.580000,-344436.650300,57175.773220,NaN,7.274589e-09,0.0,NaN,7.274589e-09,57451.424700


ethanolAA


,potential_energy,tot_vdw_energy,tail_energy,tot_electrostatics,short_range_electrostatics,long_range_electrostatics,tot_pair_energy,bonds_energy,angles_energy,dihedrals_energy,tot_bonded_energy,intramolecular_energy,intermolecular_energy
lammps-VU,31409.503929,19408.270670,-417.875752,3575.331283,81265.612659,-77690.281376,22983.601953,0.000060,7233.270516,1192.631400,8425.901976,NaN,NaN
hoomd,31409.638694,19363.949308,-417.875752,3575.466991,-12484.323747,16059.790738,19408.270670,0.000060,7233.270511,1192.630492,0.000000,0.000000,0.000000
gromacs,31409.148438,19408.189564,-417.956024,3575.072021,-14791.646484,1403.370850,22983.261585,0.000063,7233.265625,1192.620605,8425.886293,NaN,NaN
mcccs,31409.519803,19408.405387,-417.878592,3575.159662,NaN,NaN,22983.565049,0.000000,7233.316278,1192.638475,8425.954754,NaN,NaN
gomc,31409.560443,22407.883079,-417.875783,575.785432,0.000000,0.000000,22825.758862,0.000062,7233.271372,1192.620480,0.000000,11469.829714,19781.821051
cassandra,31409.469000,19408.114324,-417.876220,3575.341600,95647.914200,-92072.572600,22983.455924,NaN,7233.247800,1192.765600,NaN,11469.911144,20357.434400


### Note for GOMC energy
- `INTRA (NB)` include short range electrostatic --> got included in `tot_vdw_energy`
- Short range electrostatic is not part of `tot_electrostatics` 

In [21]:
potential_energy_table = dict()

engines = ("lammps-VU", "hoomd", "gromacs",
                    "mcccs", "gomc", "cassandra")
molecules = ("methaneUA", "pentaneUA", "benzeneUA", "ethanolAA", "waterSPCE")

for engine in engines:
    potential_energy_table[pretty_names[engine]] = list()
    for molecule in molecules:
        potential_energy_table[pretty_names[engine]].append(
            spe_data[molecule][engine]["potential_energy"])

potential_energy_df = pd.DataFrame.from_dict(potential_energy_table,
                       orient="index",
                       columns=[pretty_molecules[molecule] for molecule in molecules])

potential_energy_df.to_csv("csvs/potential_energy.csv")


In [22]:
electrostatics_table = dict()

engines = ("lammps-VU", "hoomd", "gromacs",
                    "mcccs", "gomc", "cassandra")
molecules = ("methaneUA", "pentaneUA", "benzeneUA", "ethanolAA", "waterSPCE")

for engine in engines:
    electrostatics_table[pretty_names[engine]] = list()
    for molecule in molecules:
        electrostatics_table[pretty_names[engine]].append(
            spe_data[molecule][engine]["tot_electrostatics"])

electrostatics_df = pd.DataFrame.from_dict(electrostatics_table,
                       orient="index",
                       columns=[pretty_molecules[molecule] for molecule in molecules])

electrostatics_df.to_csv("csvs/electrostatics_energy.csv")


In [23]:
vdw_table = dict()

engines = ("lammps-VU", "hoomd", "gromacs",
                    "mcccs", "gomc", "cassandra")
molecules = ("methaneUA", "pentaneUA", "benzeneUA", "ethanolAA", "waterSPCE")

for engine in engines:
    vdw_table[pretty_names[engine]] = list()
    for molecule in molecules:
        vdw_table[pretty_names[engine]].append(
            spe_data[molecule][engine]["tot_vdw_energy"])

vdw_df = pd.DataFrame.from_dict(vdw_table,
                       orient="index",
                       columns=[pretty_molecules[molecule] for molecule in molecules])

vdw_df.to_csv("csvs/vdw_energy.csv")
